In [ ]:
import numpy as np
from numpy.random import rand
#from FS.functionHO import Fun
import math

import pandas as pd 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from pandas import DataFrame, concat
import math
from numpy.random import rand
from sklearn.model_selection import train_test_split

# Data loading and preparation
data_df = pd.read_csv('8area_DS1.csv')


# Function to convert time series data into supervised learning format
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# Fitness evaluation function for LSTM
def evalVars_for_hrogwo(Vars):
    wa = 1
    wF = 0.8
    features = data_df.columns[1:]
    features_name = []
    features_name.append(data_df.columns[0])
    
    # Select features based on Vars binary
    for index in range(data_df.shape[1] - 1):
        if Vars[index] == 1:
            features_name.append(features[index])
    
    # Get selected features subset
    values = data_df[features_name]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)
    
    # Convert time series to supervised learning format
    reframed = series_to_supervised(scaled, 1, 1)
    reframed.drop(reframed.columns[[i for i in range(int(reframed.shape[1] / 2) + 1, reframed.shape[1])]], axis=1, inplace=True)

    # Split train and test
    values = reframed.values
    n_train_hours = int(reframed.shape[0] * 0.7)
    train = values[:n_train_hours, :]
    test = values[n_train_hours:, :]

    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]

    # Reshape data for LSTM
    train_X_LSTM = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X_LSTM = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    # Build LSTM model
    model = Sequential()
    model.add(LSTM(100, input_shape=(train_X_LSTM.shape[1], train_X_LSTM.shape[2])))
    model.add(Dense(1))
    
    # Compile model
    learning_rate = 0.0005
    decay_rate = learning_rate / 100
    momentum = 0.3
    sgd = SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    model.compile(loss='mse', optimizer=sgd)

    # Train model
    model.fit(train_X_LSTM, train_y, epochs=100, batch_size=128, validation_data=(test_X_LSTM, test_y), verbose=0, shuffle=False)

    # Predict and calculate RMSE
    yhat = model.predict(test_X_LSTM)
    yhat = yhat.reshape(len(yhat), 1)
    test_y = test_y.reshape(len(test_y), 1)

    rmse = math.sqrt(mean_squared_error(yhat, test_y))
    
    # Return fitness
    f = rmse * wa + ((len(features_name) - 1)/(len(data_df) - 1)) * wF
    return f



def init_position(lb, ub, N, dim):
    X = np.zeros([N, dim], dtype='float')
    for i in range(N):
        for d in range(dim):
            X[i, d] = lb[0, d] + (ub[0, d] - lb[0, d]) * rand()
    return X


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def single_binary_conversion(X, thres, dim):
    Xbin = np.zeros([1, dim], dtype='int')
    for d in range(dim):
        s = X[d]
        if s > thres:
            Xbin[0, d] = 1
        else:
            Xbin[0, d] = 0
    return Xbin


def binary_conversion(X, thres, N, dim):
    Xbin = np.zeros([N, dim], dtype='int')
    for i in range(N):
        for d in range(dim):
            s = X[i, d]
            if s > thres:
                Xbin[i, d] = 1
            else:
                Xbin[i, d] = 0

    return Xbin


def boundary(x, lb, ub):
    if x < lb:
        x = lb
    if x > ub:
        x = ub

    return x


def hrogwo(xtrain, ytrain, opts):
    # Parameters
    ub = 1
    lb = 0
    thres = 0.5
    N = opts['N']
    max_iter = opts['T']

    # Dimension
    dim = np.size(xtrain, 1)
    if np.size(lb) == 1:
        ub = ub * np.ones([1, dim], dtype='float')
        lb = lb * np.ones([1, dim], dtype='float')

    # Initialize position
    X = init_position(lb, ub, N, dim)

    # Binary conversion
    Xbin = binary_conversion(X, thres, N, dim)

    # Fitness at first iteration
    fit = np.zeros([N, 1], dtype='float')
    Xalpha = np.zeros([1, dim], dtype='float')
    Xbeta = np.zeros([1, dim], dtype='float')
    Xdelta = np.zeros([1, dim], dtype='float')
    Falpha = float('inf')
    Fbeta = float('inf')
    Fdelta = float('inf')

    X_i_GWO = np.zeros([1, dim], dtype='float')
    X_i_DLH = np.zeros([1, dim], dtype='float')
    X_cross = np.zeros([1, dim], dtype='float')

    # Replace Fun function calls with evalVars_for_hrogwo
    for i in range(N):
        fit[i, 0] = evalVars_for_hrogwo(Xbin[i, :])  # Using evalVars_for_hrogwo instead of Fun
        if fit[i, 0] < Falpha:
            Xalpha[0, :] = X[i, :]
            Falpha = fit[i, 0]

        if fit[i, 0] < Fbeta and fit[i, 0] > Falpha:
            Xbeta[0, :] = X[i, :]
            Fbeta = fit[i, 0]

        if fit[i, 0] < Fdelta and fit[i, 0] > Fbeta and fit[i, 0] > Falpha:
            Xdelta[0, :] = X[i, :]
            Fdelta = fit[i, 0]

    curve = np.zeros([1, max_iter], dtype='float')
    t = 0

    curve[0, t] = Falpha.copy()
    print("Iteration:", t + 1)
    print("Best (HRO-GWO):", curve[0, t])
    t += 1

    while t < max_iter:
        w = 0.01
        a = 2 * ((1 / 4) ** (t / max_iter)) * np.exp(-(w * (max_iter - t) / max_iter))
        for i in range(N):
            Neibor = []
            p = rand()
            for d in range(dim):
                # Parameter C
                C1 = 2 * rand()
                C2 = 2 * rand()
                C3 = 2 * rand()
                # Compute Dalpha, Dbeta & Ddelta
                Dalpha = abs(C1 * Xalpha[0, d] - X[i, d])
                Dbeta = abs(C2 * Xbeta[0, d] - X[i, d])
                Ddelta = abs(C3 * Xdelta[0, d] - X[i, d])
                # Parameter A
                A1 = 2 * a * rand() - a
                A2 = 2 * a * rand() - a
                A3 = 2 * a * rand() - a

                X1 = Xalpha[0, d] - A1 * Dalpha
                X2 = Xbeta[0, d] - A2 * Dbeta
                X3 = Xdelta[0, d] - A3 * Ddelta
                # Update wolf
                X_i_GWO[0, d] = (X1 + X2 + X3) / 3
                X_i_GWO[0, d] = boundary(X_i_GWO[0, d], lb[0, d], ub[0, d])

                if p <= 0.5:
                    A = 2 * a * rand() - a
                    if abs(A) > 0.8:
                        r1 = rand()
                        r2 = rand()
                        r3 = rand()
                        X_cross[0, d] = (r1 * Xalpha[0, d] + r2 * Xbeta[0, d] + r3 * Xdelta[0, d]) / (r1 + r2 + r3)
                    else:
                        X_cross[0, d] = X1
                    if rand() < 0.3:

                        X_cross[0, d] = X[i, d]
                    X_cross[0, d] = boundary(X_cross[0, d], lb[0, d], ub[0, d])

                else:
                    random_number = np.random.randint(low=0, high=N)
                    neighbor = X[random_number]
                    r3 = np.random.random()
                    X_cross[0, d] = r3 * (Xalpha[0, d] - neighbor[d]) + X[i, d]
                    X_cross[0, d] = boundary(X_cross[0, d], lb[0, d], ub[0, d])

            X_cross_bin = single_binary_conversion(X_cross[0, :], thres, dim)
            X_i_GWO_bin = single_binary_conversion(X_i_GWO[0, :], thres, dim)
            F_i_cro = evalVars_for_hrogwo(X_cross_bin[0, :])  # Using evalVars_for_hrogwo instead of Fun
            F_i_GWO = evalVars_for_hrogwo(X_i_GWO_bin[0, :])  # Using evalVars_for_hrogwo instead of Fun

            f = []
            R = np.sqrt(np.sum((X[i, :] - X_i_GWO[0, :]) ** 2))
            for j in range(N):
                if i != j:
                    R_n = np.sqrt(np.sum((X[i, :] - X[j, :]) ** 2))
                    if R_n <= R:
                        Neibor.append(X[j, :].tolist())
                        f.append(fit[j, 0].tolist())
            Neibor = np.array(Neibor, dtype=float)
            f = np.array(f, dtype=float)


            if len(Neibor) != 0:
                for d in range(dim):
                    k1 = np.random.randint(low=0, high=len(Neibor))
                    k2 = np.random.randint(low=0, high=N)

                    if (f[k1] < fit[k2, 0]):

                        X_i_DLH[0, d] = X[i, d] + rand() * (Neibor[k1, d] - X[k2, d])
                    else:
                        X_i_DLH[0, d] = X[i, d] - rand() * (Neibor[k1, d] - X[k2, d])

                    X_i_DLH[0, d] = boundary(X_i_DLH[0, d], lb[0, d], ub[0, d])
                X_i_DLH_bin = single_binary_conversion(X_i_DLH[0, :], thres, dim)
                F_i_DLH = evalVars_for_hrogwo(X_i_DLH_bin[0, :])  # Using evalVars_for_hrogwo instead of Fun
                if (F_i_GWO < F_i_DLH) and (F_i_GWO < F_i_cro):
                    X[i, :] = X_i_GWO[0, :]
                    fit[i, 0] = F_i_GWO

                elif (F_i_cro < F_i_DLH) and (F_i_cro < F_i_GWO):

                    X[i, :] = X_cross[0, :]
                    fit[i, 0] = F_i_cro

                else:
                    X[i, :] = X_i_DLH[0, :]
                    fit[i, 0] = F_i_DLH

            else:
                if F_i_GWO < F_i_cro:
                    X[i, :] = X_i_GWO[0, :]
                    fit[i, 0] = F_i_GWO

                else:
                    X[i, :] = X_cross[0, :]
                    fit[i, 0] = F_i_cro


            if fit[i, 0] < Falpha:
                Xalpha[0, :] = X[i, :]
                Falpha = fit[i, 0]

            if fit[i, 0] < Fbeta and fit[i, 0] > Falpha:
                Xbeta[0, :] = X[i, :]
                Fbeta = fit[i, 0]

            if fit[i, 0] < Fdelta and fit[i, 0] > Fbeta and fit[i, 0] > Falpha:
                Xdelta[0, :] = X[i, :]
                Fdelta = fit[i, 0]

        curve[0, t] = Falpha
        print("Iteration:", t + 1)
        print("Best (HRO-GWO):", curve[0, t])
        t += 1

    # Best feature subset
    Gbin = binary_conversion(Xalpha, thres, 1, dim)
    Gbin = Gbin.reshape(dim)
    pos = np.asarray(range(0, dim))
    sel_index = pos[Gbin == 1]
    num_feat = len(sel_index)

    # 打印选择的最优特征子集
    print("Selected Feature Subset Indices:")
    print(sel_index)

    # Create dictionary
    hrogwo_data = {'sf': sel_index, 'c': curve, 'nf': num_feat, 'fitness': curve[0, max_iter - 1]}

    return hrogwo_data


# 数据准备
y = data_df.iloc[:, 0].values  # 选择第一列作为目标变量
X = data_df.iloc[:, 1:].values  # 选择从第二列开始到最后一列作为特征变量
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

# HO算法的参数设置
opts = {'N': 30, 'T': 50}  # 种群大小和最大迭代次数

# 执行HRO-GWO算法S
result = hrogwo(xtrain, ytrain, opts)

# 打印结果
print("Selected feature indices:", result['sf'])  # 打印最优特征子集的索引
print("Best fitness value:", result['fitness'])  # 打印最佳适应度值
print("Number of selected features:", result['nf'])  # 打印选择的特征数量


